In [221]:
import pandas as pd
from os import listdir
import unidecode

In [222]:
additional_info = pd.read_csv('additional_csv/addinfo.csv')
renta_pv = pd.read_csv('additional_csv/rentas_PV.csv',  encoding='ISO-8859-1', sep=';')
rentas_espanya = pd.read_csv('additional_csv/rentas_espanya.csv').rename(columns={'Unnamed: 0':'provincia'})

In [223]:
# Remove code, accencts and all names to lower case
additional_info['poblacion'] = additional_info['poblacion'].apply(lambda x: unidecode.unidecode(' '.join(x.split(' ')[1:])).lower())

rentas_espanya['provincia'] = rentas_espanya['provincia'].apply(lambda x: unidecode.unidecode(''.join(x.split('-')[0])).lower())

renta_pv['ambitos territoriales'] = renta_pv['ambitos territoriales'].apply(lambda x: unidecode.unidecode(x).lower())

In [224]:
all_files = []
path = 'province_houses/'
for filename in listdir(path):
    if 'houses' in filename and 'csv' in filename:
        provincia = unidecode.unidecode(' '.join(filename.split('_')[1:len(filename.split('_')):1]).split('.')[0])
        renta_media = rentas_espanya[rentas_espanya['provincia'].str.contains(provincia)]['Renta disponible media']
        if renta_media.empty:
            renta_media = renta_pv[(renta_pv['ambitos territoriales'].str.contains(provincia)) & (renta_pv['tipo de renta'] == 'Renta total')]['2016']
        if renta_media.empty:
            renta_media = None
        else:
            renta_media = renta_media.values[0]
            
        valor_empresa_muni = pd.to_numeric(additional_info[(additional_info['poblacion'].str.contains(provincia)) & (additional_info['source'] == 'Empresas')]['valor'].values[0].replace('.', ''))
        valor_poblacion_muni = pd.to_numeric(additional_info[(additional_info['poblacion'].str.contains(provincia)) & (additional_info['source'] == 'Poblacion')]['valor'].values[0].replace('.', ''))
        
        valor_empresa_nacional = pd.to_numeric(additional_info[(additional_info['poblacion'].str.contains('nacional')) & (additional_info['source'] == 'Empresas')]['valor'].values[0].replace('.', ''))
        valor_poblacion_nacional = pd.to_numeric(additional_info[(additional_info['poblacion'].str.contains('nacional')) & (additional_info['source'] == 'Poblacion')]['valor'].values[0].replace('.', ''))

        
        df = pd.read_csv(path + filename)
        df['source'] = filename.split('.')[0]
        df['number_of_companies_prov'] = valor_empresa_muni
        df['population_prov'] = valor_poblacion_muni
        df['companies_prov_vs_national_%'] = ((valor_empresa_muni / valor_empresa_nacional) * 100).round(2)
        df['population_prov_vs_national_%'] = ((valor_poblacion_muni / valor_poblacion_nacional) * 100).round(2)
        df['renta_media_prov'] = renta_media

        
        all_files.append(df)

In [225]:
result = pd.concat(all_files)

In [226]:
del result['source']
result.to_csv('spanish_houses.csv', index=False)

In [227]:
result

,ad_description,ad_last_update,air_conditioner,balcony,bath_num,built_in_wardrobe,chimney,condition,construct_date,energetic_certif,...,room_num,storage_room,swimming_pool,terrace,unfurnished,number_of_companies_prov,population_prov,companies_prov_vs_national_%,population_prov_vs_national_%,renta_media_prov
0,Precio chalet individual en la localidad de Ab...,Anuncio actualizado el 27 de marzo,0,0,2,0,0,segunda mano/buen estado,NaN,NaN,...,4,0,0,1,NaN,19147,328868,0.57,0.70,19889
1,"Atico de 80m2, para entrar a vivir, con salón ...",más de 5 meses sin actualizar,0,0,2,0,0,segunda mano/buen estado,2006,no indicado,...,3,1,0,0,NaN,19147,328868,0.57,0.70,19889
2,B/ Etxaguen. Casa de reciente construcción con...,más de 5 meses sin actualizar,0,0,3,0,0,segunda mano/buen estado,NaN,no indicado,...,4,1,0,1,NaN,19147,328868,0.57,0.70,19889
3,Se vende vivienda en abornikano (ayuntamiento ...,más de 5 meses sin actualizar,0,1,1,1,1,segunda mano/buen estado,NaN,en trámite,...,4,1,0,1,NaN,19147,328868,0.57,0.70,19889
4,Negociables.,más de 5 meses sin actualizar,0,0,1,0,0,segunda mano/buen estado,NaN,no indicado,...,2,1,1,1,NaN,19147,328868,0.57,0.70,19889
5,Magnifico chalet unifamiliar situado en Ametza...,más de 5 meses sin actualizar,0,0,2,0,0,segunda mano/buen estado,NaN,no indicado,...,4,0,0,0,NaN,19147,328868,0.57,0.70,19889
6,"Piso luminoso y céntrico, todo exterior, orien...",más de 5 meses sin actualizar,0,0,2,0,0,segunda mano/buen estado,NaN,no indicado,...,3,1,0,1,NaN,19147,328868,0.57,0.70,19889
7,Se vende caserio restaurado de piedra a los pi...,más de 5 meses sin actualizar,0,0,3,0,1,segunda mano/buen estado,NaN,no indicado,...,7,0,0,0,NaN,19147,328868,0.57,0.70,19889
8,"distribución:,2 vecinos en el edificio",más de 5 meses sin actualizar,0,0,2,0,0,segunda mano/buen estado,NaN,no indicado,...,3,1,0,0,NaN,19147,328868,0.57,0.70,19889
9,Precio negociable. Una casa en un entorno de t...,más de 5 meses sin actualizar,0,1,3,1,0,segunda mano/buen estado,NaN,NaN,...,4,0,0,1,NaN,19147,328868,0.57,0.70,19889
